In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\ramak\\OneDrive\\Desktop\\P2\\Birds_Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class TrainingModelConfig:
    root_dir: Path
    trained_model_dir: Path
    train_data: Path
    test_data: Path
    validation_data: Path
    epochs: int
    batch_size: int
    saved_model_dir: Path
    input_shape: int

In [4]:
from src.BirdImageClassification.constants import *
from src.BirdImageClassification.utils.common import read_yaml,create_directory, save_json
from src.BirdImageClassification import logger

True 
 True


In [5]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directory([self.config.main_dir])

    def training_config(self) -> TrainingModelConfig:
        config = self.config.training_model
        create_directory([config.root_dir])

        prepare_training_model = TrainingModelConfig(
            root_dir= Path(config.root_dir),
            trained_model_dir= Path(config.trained_model_dir),
            saved_model_dir= Path(config.saved_model_dir),
            train_data= Path(config.train_data),
            validation_data= Path(config.validation_data),
            test_data= Path(config.test_data),
            batch_size= self.params.BATCH_SIZE,
            epochs= self.params.EPOCHS,
            input_shape= self.params.INPUT_SHAPE
        )

        return prepare_training_model

In [6]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator


[2023-12-22 12:18:30,893: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
class TrainingModel:
    def __init__(self, config = TrainingModelConfig):
        self.config = config

    def load_model(self):
        self.model = tf.keras.models.load_model(self.config.saved_model_dir)

    
    def prepare_training_model(self):
        early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=30,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )

        validation_datagen = ImageDataGenerator(rescale=1./255)
        test_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(self.config.train_data,
                                                            target_size = (self.config.input_shape, self.config.input_shape),
                                                            batch_size = self.config.batch_size,
                                                            class_mode = 'categorical')
        
        validation_generator = validation_datagen.flow_from_directory(self.config.validation_data,
                                                            target_size = (self.config.input_shape, self.config.input_shape),
                                                            batch_size = self.config.batch_size,
                                                            class_mode = 'categorical')
        
        test_generator = test_datagen.flow_from_directory(self.config.test_data,
                                                            target_size = (self.config.input_shape, self.config.input_shape),
                                                            batch_size = self.config.batch_size,
                                                            class_mode = 'categorical')
        

        self.model.fit(train_generator,
                       epochs = self.config.epochs,
                    #    steps_per_epoch = len(train_generator),
                       validation_data = validation_generator,
                    #    validation_steps = len(validation_generator) * 0.25,
                       verbose = 1,
                       callbacks = [early_stopping])
        

        scores = self.model.evaluate(
                                        validation_generator,
                                        batch_size=self.config.batch_size
                                    )

        score = {"loss": scores[0], "accuracy": scores[1]}
        save_json(path=Path("scores.json"), data=score)

        self.save_model(path=self.config.trained_model_dir, model=self.model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [8]:
try:
    config_manager = ConfigurationManager()  # Create an instance of ConfigurationManager
    training_config = config_manager.training_config()
    training = TrainingModel(config = training_config)
    training.load_model()
    # training.data_preparation()
    training.prepare_training_model()
    
except Exception as e:
    raise e

[2023-12-22 12:18:32,784: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\Birds_Classification\config\config.yaml loaded]
[2023-12-22 12:18:32,784: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\Birds_Classification\params.yaml loaded]
[2023-12-22 12:18:32,784: INFO: common: Created directory at: artifacts]
[2023-12-22 12:18:32,784: INFO: common: Created directory at: artifacts/training_model]
Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Epoch 1/5
2645/2645 [==============================] - 2531s 954ms/step - loss: 3.4082 - accuracy: 0.2737 - val_loss: 1.8118 - val_accuracy: 0.5326
Epoch 2/5
2645/2645 [==============================] - 2454s 928ms/step - loss: 2.7560 - accuracy: 0.3715 - val_loss: 1.4718 - val_accuracy: 0.6213
Epoch 3/5
2645/2645 [==============================] - 2326s 879ms/step - loss: 2.6098 - accuracy: 0.4009 - val_loss: 1.3338 - val_accuracy: 0.6583
E

C:\Users\ramak\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
